In [1]:
import numpy as np
import math
import pygame
import time
import threading

pygame-ce 2.3.2 (SDL 2.26.5, Python 3.10.7)


In [4]:
class RadialCell:
    def __init__(self, _pos, _color, _id):
        self.pos = _pos
        self.color = _color
        self.id = _id
        
    def change_color(self, _scale):
        pert = ((np.random.rand(3) * 2.0) - 1.0) * _scale
        self.color = np.clip(self.color + pert, 0.0, 1.0)

    def move(self, _scale):
        pert = ((np.random.rand(3) * 2.0) - 1.0) * _scale
        pert[2] = 0.0
        self.pos += pert

class RadialAutomata:
    def __init__(self, _radius, _rate=0.01, _color_scale=0.1, _move_scale=0.8):
        self.radius = _radius
        self.rate = _rate
        self.id_count = 0
        self.color_scale = _color_scale
        self.move_scale = _move_scale
        
        # * init first cell
        self.cells = []
        self.cells.append(RadialCell(np.array([0.0, 0.0, 0.0]), np.array([0.5, 0.5, 0.5]), 0))
        self.id_count += 1
        
    def update(self):
        # * stochastic update
        stochastic_mask = np.random.rand(len(self.cells)) > self.rate
        active_cells =  np.argwhere(stochastic_mask).flatten()
        
        # * perception step
        perception = self.percieve(active_cells)
        
        # * selection step
        actions = self.select(perception)
        
        # * each active cell performs its action
        self.perform(active_cells, actions, perception)
        
    def percieve(self, _active_cells):
        perception = {}

        for cell in _active_cells:
            cells_in_rad = []
            for neighbor in self.cells:
                dis = math.dist(self.cells[cell].pos, neighbor.pos)
                if dis < self.radius and self.cells[cell].id != neighbor.id:
                    dir = neighbor.pos - self.cells[cell].pos
                    norm = np.linalg.norm(dir)
                    dir = dir/norm
                    cells_in_rad.append({'dis': dis, 'dir': dir, 'color': neighbor.color})
            perception[cell] = cells_in_rad
        return perception
    
    def select(self, _perception):
        # * types of actions:
        #       - [0] dupicate
        #       - [1] destruct
        #       - [2] change color
        #       - [3] move
        actions = [] 
        for p in _perception:
            # * for now, select action at random
            # * TODO train nn to take perception and select action (?)
            actions.append(np.random.randint(0, 4))
        return actions
    
    def perform(self, _cells, _actions, _perception):
        # * perform non-size-changing actions:
        for i in range(len(_cells)):
            cell = _cells[i]
            action = _actions[i]
            neighbors = len(_perception[cell])
            # * change a cell's color slightly
            if action == 2 and neighbors <= 4:
                self.cells[cell].change_color(self.color_scale)
            # * move a cell's position slightly
            elif action == 3:
                self.cells[cell].move(self.move_scale)
                
        # * perform size-changing actions:
        new_cells = self.cells.copy()
        for i in range(len(_cells)):
            cell = _cells[i]
            action = _actions[i]
            neighbors = len(_perception[cell])
            # * duplicate cell in random direction
            if action == 0:
                dup_pos = np.array(self.cells[cell].pos) + ((np.random.rand(3) * 2.0) - 1) * self.move_scale
                dup_pos[2] = 0.0
                new_cells.append(RadialCell(dup_pos, self.cells[cell].color, self.id_count))
                self.id_count += 1
            # * destroy cell (if action dictates or over-populated)
            elif neighbors > 5:
                new_cells.remove(self.cells[cell])
        self.cells = new_cells

In [5]:
_WINDOW_BG_COLOR_ = (255, 255, 255)
_WINDOW_TEXT_COLOR_ = (0, 0, 0)
_SIZE_ = 512

pygame.init()
pygame.display.set_caption('radial automata simulation')

window = pygame.display.set_mode((_SIZE_, _SIZE_))
automata = RadialAutomata(1.0)

# * create clock for fps
clock = pygame.time.Clock()
delta_time = 0

# * setup text rendering
font_size = 20
my_font = pygame.font.SysFont('consolas', font_size)
fps_surface = my_font.render(f'{clock.get_fps() :.0f}', False, _WINDOW_TEXT_COLOR_)
cells_sureface = my_font.render(f'dots: {len(automata.cells)}', False, _WINDOW_TEXT_COLOR_)

running = True 
# def run_sim(_delay):
#     time.sleep(_delay)
#     while running:
#         automata.update()

# * start forward worker
#sim = threading.Thread(target=run_sim, args=[1], daemon=True)
#sim.start()

while running:
    # * close application
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                break
            if event.key == pygame.K_RETURN:
                automata.update()
    
    # * draw tensor to window
    window.fill(_WINDOW_BG_COLOR_)
    
    # * blip cells
    for cell in automata.cells:
        color = np.array(cell.color * 255.0, dtype=int)
        pos = (cell.pos*8) + (_SIZE_/2, _SIZE_/2, 0.0)
        pygame.draw.circle(window, color, pos[:2,], 5, 5) #(r, g, b) is color, (x, y) is center, R is radius and w is the thickness of the circle border.
    
    # * calculate fps
    delta_time = clock.tick()
    fps_surface = my_font.render(f'{clock.get_fps() :.0f}', False, _WINDOW_TEXT_COLOR_)
    window.blit(fps_surface, (0, 0))

    # * show number of cells
    cells_sureface = my_font.render(f'dots: {len(automata.cells)}', False, _WINDOW_TEXT_COLOR_)
    window.blit(cells_sureface, (0, _SIZE_-font_size))
    
    # * flip it!
    pygame.display.flip()

# * quit it!
pygame.quit()
# sim.join()

UnboundLocalError: local variable 'cell' referenced before assignment